In [1]:
import torch
import torchvision
import torchvision.transforms as transforms #数据转换操作
from torchvision.transforms import ToPILImage

ModuleNotFoundError: No module named 'torch'

# 预处理

In [ ]:
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# 数据集

In [ ]:
train_set = torchvision.datasets.CIFAR10(root = "/home/files/dataset/cifar-10-python/",
                                         train = False,
                                         transform = transforms)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set,
                                          batch_size = 64,
                                          shuffle = True,
                                          num_workers = 2)

In [ ]:
test_set = torchvision.datasets.CIFAR10(root = "/home/files/dataset/cifar-10-python/",
                                       train = False,
                                       transform = transforms)

In [ ]:
test_loader = torch.utils.data.DataLoader(test_set,
                                         batch_size = 64,
                                         shuffle = False,
                                         num_workers = 2)

In [ ]:
classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

datasets对象是一个数据集，可以按下标访问，返回（data, label）的数据

In [ ]:
(data, label) = train_set[100]
print("data:", data)
print("label:", classes[label])
print(train_set)

DataLoader是一个可迭代对象，将dataset返回的每一条数据拼接为一个batch，并提供多线程加速优化和数据打乱等操作

# 定义网络结构

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) #分别代表in_channel,out_channel,kernel_size
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120) #full connection layer
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x): #注意forward有输入，在nn.Module的子类中定义了forward类后，backward函数就会利用Autograd自动实现
        #print("init x size:", x.size())
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        #print("after 1 conv:", x.size())
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        #print("after 2 convs:",x.size())
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
print(net)

# 定义损失函数和优化器

In [ ]:
from torch import optim
from torch.autograd import Variable
criterion = nn.CrossEntropyLoss()
# criterion = nn.CrossEntropyLoss().cuda() 
optimizer = optim.SGD(net.parameters(), lr = 0.01, momentum = 0.9)


# 训练网络
输入数据====>前向&反向传播====>更新参数

In [ ]:
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        # 输入数据
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels) #转换为Variable类型
        
        #梯度清零
        optimizer.zero_grad()
        
        #forward&backward
        output = net(inputs)
        loss = criterion(output, labels)
        loss.backward()
        
        #更新参数
        optimizer.step()
        
        #log
        running_loss += loss
        if i%20 == 0:
            print('[%d, %5d] loss :%.3f' % (epoch+1, i, running_loss/20)) #loss/step
            running_loss = 0
            
print("finished training!")
        
        

# 测试

## real label

In [ ]:
dataiter = iter(test_loader)
images, labels = dataiter.next()
print("实际的label：", ' '.join('%5s'%classes[labels[j]] for j in range(64)))
show = ToPILImage()
show(torchvision.utils.make_grid(images/2-0.5)).resize((400,400))

## predicted label

In [ ]:
outputs = net(images)
_, predicted = torch.max(outputs.data, 1) #value, index
print("预测结果：", ' '.join('%5s'%classes[predicted[j]] for j in range(64)))


In [ ]:
print("准确率：",torch.sum(labels == predicted).numpy()/64.0)

## 在整个数据集上的效果

In [ ]:
correct = 0
total = 0
for data in test_loader:
    images, labels = data
    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
print("准确率为：%d %%" % (100*correct/total) )

# 在GPU上进行训练

In [ ]:
for epoch in range(200):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        # 输入数据
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels) #转换为Variable类型
        
        #cuda 注意在使用cuda时，需要将优化器In[18]也进行设置
        inputs = inputs.cuda()
        labels = labels.cuda()
        net.cuda()
        criterion = criterion.cuda()
        optimizer = optim.SGD(net.parameters(), lr = 0.01, momentum = 0.9)
        #梯度清零
        optimizer.zero_grad()
        
        #forward&backward
        output = net(inputs)
        loss = criterion(output, labels)
        loss.backward()
        
        #更新参数
        optimizer.step()
        
        #log
        running_loss += loss
        if i%20 == 0:
            print('[%d, %5d] loss :%.3f' % (epoch+1, i, running_loss/20)) #loss/step
            running_loss = 0
            
print("finished training!")

In [ ]:
correct = 0
total = 0
for data in test_loader:
    images, labels = data
    outputs = net(images.cuda())
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
print("准确率为：%d %%" % (100*correct/total) )